In [1]:
import pandas as pd
import requests
from requests import request
from bs4 import BeautifulSoup
from multiprocessing.dummy import Pool as ThreadPool
import functools
import time
from tqdm.notebook import tqdm
import sys
from itertools import chain
import re
import functools
import pymorphy2

In [2]:
import pandas as pd 
import datetime
import numpy as np 
import matplotlib.pyplot as plt 

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import bs4
from multiprocessing import Pool, Lock, Value
from time import sleep
from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate
import base64
import numpy as np
import zlib
from itertools import product
import re

In [4]:
import pymorphy2  # $ pip install pymorphy2
  
def pos(word, morth=pymorphy2.MorphAnalyzer()):
    "Return a likely part of speech for the *word*."""
    return morth.parse(word)[0].tag.POS
  
words = "Однако я так и не смог закончить".split()
functors_pos = {'INTJ', 'PRCL', 'CONJ', 'PREP'}  # function words
# p2 = [word for word in p1 if (pos(word) not in functors_pos and len(word)>1)]
# -> я смог закончить

In [5]:
list_mest = ['я', 'ты', 'он', 'она', 'оно', 'мы',
             'вы', 'они','мой', 'твой', 'свой', 'ваш', 'наш', 'его', 'её', 'их',
            'все','быть','может', 'можно', 'очень','нужно','будет','уже','могут',
            'кто', 'что', 'какой', 'чей', 'где', 'который', 'откуда', 'сколько', 'каковой', 'каков', 'зачем',
            'меня','мне','того', 'этого', 'этот','ее','вы','более', 'было', 'вам', 'все', 'вы',  
             'которые', 'нет',  'том', 'этом',
            'вас', 'вообще', 'всего', 'вы',
            'себе', 'себя', 'ему', 'тебе', 'тебя','был']

In [6]:
morph = pymorphy2.MorphAnalyzer()

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [8]:


def batch_generator(X, y, shuffle=True, batch_size=1):
    """
    Гератор новых батчей для обучения
    X          - матрица объекты-признаки
    y_batch    - вектор ответов
    shuffle    - нужно ли случайно перемешивать выборку
    batch_size - размер батча ( 1 это SGD, > 1 mini-batch GD)
    Генерирует подвыборку для итерации спуска (X_batch, y_batch)
    """
    ind = np.array(range(y.shape[0]), dtype = int)
    if(shuffle):
        ind = np.random.permutation(y.shape[0])
    for i in range(len(y)//batch_size):
        X_batch = X[ind[i*batch_size:(i+1)*batch_size],:]
        y_batch = y[ind[i*batch_size:(i+1)*batch_size]]
        yield (X_batch, y_batch)

In [9]:
from sklearn.base import BaseEstimator, ClassifierMixin


def sigmoid(x):
    """
    Вычисляем значение сигмоида.
    X - выход линейной модели
    """
    # Your code Here
    sigm_value_x = 1/(1 + np.exp(-(x)))
    return sigm_value_x


class MySGDClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, batch_generator, C=1, alpha=0.01,
                 max_epoch=10, model_type='lin_reg', batch_size=1, th=0.5):
        """
        batch_generator -- функция генератор, которой будем создавать батчи
        C - коэф. регуляризации
        alpha - скорость спуска
        max_epoch - максимальное количество эпох
        model_type - тим модели, lin_reg или log_reg
        """
        self.th = th
        self.C = C
        self.alpha = alpha
        self.max_epoch = max_epoch
        self.batch_generator = batch_generator
        self.errors_log = {'iter' : [], 'loss' : []}  
        self.model_type = model_type
        self.batch_size = batch_size

    def calc_loss(self, X_batch, y_batch):
        """
        Считаем функцию потерь по батчу 
        X_batch - матрица объекты-признаки по батчу
        y_batch - вектор ответов по батчу
        Не забудте тип модели (линейная или логистическая регрессия)!
        """
        if (self.model_type == 'lin_reg'):
            loss = 1 / y_batch.shape[0] * ((np.dot(X_batch, self.weights.reshape(-1, 1)) - y_batch) ** 2).sum() 
            + 1 / self.C * ((self.weights * self.weights).sum())
            return loss
        else:
            sigm = np.arange(y_batch.shape[0], dtype = float)
            for i in range(y_batch.shape[0]):
                sigm[i] = sigmoid((X_batch[i, : ] * self.weights).sum())
            loss = - (y_batch * np.log(sigm) + (1 - y_batch) * np.log(1 - sigm)).sum() / y_batch.shape[0] 
            + 1 / self.C * ((self.weights * self.weights).sum())
            return loss

    def calc_loss_grad(self, X_batch, y_batch):
        """
        Считаем  градиент функции потерь по батчу (то что Вы вывели в задании 1)
        X_batch - матрица объекты-признаки по батчу
        y_batch - вектор ответов по батчу
        Не забудте тип модели (линейная или логистическая регрессия)!
        """
        if (self.model_type == 'lin_reg'):
            loss_grad = (2/y_batch.shape[0])*np.dot((np.dot(X_batch, self.weights.reshape(-1,1)) 
                                        - y_batch.reshape(-1,1)).T,X_batch).sum(axis = 0) + 2/self.C*self.weights
            return loss_grad.reshape(-1)
        else:
            sigm = np.arange(y_batch.shape[0], dtype = float)
            for i in range(y_batch.shape[0]):
                sigm[i] = sigmoid((X_batch[i, : ]*self.weights).sum())
            loss_grad = np.dot(X_batch.T,(sigm - y_batch).reshape(-1,1)).reshape(self.weights.shape[0])
            + 2/self.C*self.weights
            return loss_grad

    def update_weights(self, new_grad):
        """
        Обновляем вектор весов
        new_grad - градиент по батчу
        """
        self.weights -= self.alpha *new_grad

    def fit(self, X, y):
        '''
        Обучение модели
        X - матрица объекты-признаки
        y - вектор ответов
        '''
        # Нужно инициализровать случайно веса
        X = np.hstack((np.ones((X.shape[0], 1)), X))
        self.weights = np.random.random(X.shape[1])
        for n in range(0, self.max_epoch):
            new_epoch_generator = self.batch_generator(X, y, batch_size = self.batch_size)
            for batch_num, new_batch in enumerate(new_epoch_generator):
                X_batch = new_batch[0]
                y_batch = new_batch[1]
                batch_loss = self.calc_loss(X_batch, y_batch)
                batch_grad = self.calc_loss_grad(X_batch, y_batch)
                self.update_weights(batch_grad)
                # Подумайте в каком месте стоит посчитать ошибку для отладки модели
                # До градиентного шага или после
                self.errors_log['iter'].append(batch_num)
                self.errors_log['loss'].append(batch_loss)
        return self

    def predict(self, X):
        '''
        Предсказание класса
        X - матрица объекты-признаки
        Не забудте тип модели (линейная или логистическая регрессия)!
        '''
        X = np.hstack((np.ones((X.shape[0], 1)), X))
        if (self.model_type == 'lin_reg'):
            y_hat = np.dot(X, self.weights)/np.sum(X)
        else:
            sigm = [[] for i in range(X.shape[0])]
            for i in range(X.shape[0]):
                sigm[i] = sigmoid((X[i, : ]*self.weights).sum())
            y_hat = (np.array(sigm))
        # Желательно здесь использовать матричные операции между X и весами, например, numpy.dot 
        return y_hat

In [10]:
a = 'mexoro'
print(len(a[:3]))

3


In [11]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /home/nastya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

In [13]:
russian_stopwords = stopwords.words("russian")

In [14]:
import csv
doc_to_h = {}

In [15]:
with open('first_100_mystem.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    k = 0
    for row in readCSV:
        doc_to_h[int(row[0])] = row[1].split()
#         print(row[0])
#         print(row[1])

In [22]:
import re
from pymystem3 import Mystem
m = Mystem()
doc_to_h1 = {}
for doc_id in doc_to_h.keys():
    title = doc_to_h[doc_id]
#     r = [ i.lower() for i in title[:100] if i]
#     p = [re.sub(r'[^А-я,0-9]', '', i) for i in r]
#     p1 = list(filter(lambda i: i, p))
#     cur = [word for word in p1 if (pos(word) not in functors_pos and len(word)>1)]
#     print(title)
    cur = [u.lower() for u in title if not u.lower() in list_mest]
    cur = cur[:200]
    cur = [i for i in cur if not i in russian_stopwords]
#     cur = ' '.join(cur)
#         cur = [morph.parse(o)[0].normal_form for o in cur]

    title = ''
    for i in cur:
        if len(i)>2:
            title += i + ' '
#     print(title)
#     print(doc_id)
    doc_to_h1[doc_id] = title
print (len(doc_to_h1))

28026


In [17]:
import pandas as pd
train_d11 = pd.read_csv('train_groups.csv')
traingroups_titled11 = {}
for i in range(len(train_d11)):
    new_doc = train_d11.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    title = doc_to_h1[doc_id]
#     title = [u.lower() for u in title if not u.lower() in list_mest]
#     title = [morph.parse(o)[0].normal_form for o in title]
    if doc_group not in traingroups_titled11:
        traingroups_titled11[doc_group] = []
    traingroups_titled11[doc_group].append((doc_id, title, target))

In [23]:
%%time
# вот этот кусок кода!!!!!!!!!!!!!!!!!!!

import numpy as np
import Levenshtein
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
y_train1 = []
X_train4 = []
groups_train1 = []
for new_group1 in traingroups_titled11:
    docs1 = traingroups_titled11[new_group1]
    list_data = []
    for k, (doc_id, title, target_id) in enumerate(docs1):
        title = title.strip().split()
        title1 = [u.lower() for u in title if not u.lower() in list_mest]
        list_data.append(' '.join(title1[:20]))
    vectorizer = TfidfVectorizer(max_features=20)
    X6 = vectorizer.fit_transform(list_data)
    dic = set(vectorizer.get_feature_names())
#     print(dic)
    vectorizer = CountVectorizer(max_features=20).fit_transform(list_data)
    vectores = vectorizer.toarray()
    dist = cosine_similarity(X6, X6)
    for k1, (doc_id1, title1, target_id1) in enumerate(docs1):
        y_train1.append(target_id1)
        groups_train1.append(new_group1)
        words1 = set((title1[:20]).strip().split())
        sum1 = 0
        all_dist2 = []
        all_dist4 = []
        all_dist1 = []
        for j in range(0, len(docs1)):
            if k1 == j:
                continue
            doc_id_j, title_j, target_j = docs1[j]
            words_j = set((title_j[:20]).strip().split())
            y = 0
            if (len(words1)+len(words_j)-len(words1.intersection(words_j)))!=0:
                y = len(words1.intersection(words_j))/(len(words1)+len(words_j)-len(words1.intersection(words_j)))
            y1 = Levenshtein.distance(title1[:20],title_j[:20])
            y2 = dist[k1,j]
            all_dist4.append(y2)
            all_dist2.append(y1)
            all_dist1.append(y)
        for i in sorted(all_dist1, reverse=True)[0:15]:
            sum1+=i
        sum2 = 0
        sum4 = 0
        ss = 0
        for i in sorted(all_dist2, reverse=False)[0:5]:
            sum2+=i
        for i in sorted(all_dist4, reverse=False)[0:15]:
            sum4+=i
        for i in sorted(all_dist2, reverse=False)[0:15]:
            ss+=i
        all_dist3=len(words1.intersection(dic))
        f = ('купить' in words1) or ('продать'in words1)
#         if f==1:
#             print(1)
        X_train4.append(sorted(all_dist4, reverse=False)[0:2:10]+[all_dist3,sum4,sum1,ss,len(title1.strip().split()),f] + sorted(all_dist1, \
                                reverse=True)[0:15])
X_train4 = np.array(X_train4)
y_train1 = np.array(y_train1)
groups_train1 = np.array(groups_train1)
print (X_train4.shape, y_train1.shape, groups_train1.shape)

(11690, 22) (11690,) (11690,)
CPU times: user 6.07 s, sys: 1.91 ms, total: 6.07 s
Wall time: 6.07 s


In [18]:
import re
from pymystem3 import Mystem
doc_to_t = {}
m = Mystem()
with open('docs_titles.tsv') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
#         cur = re.split(r' ',title)
        r = [ i.lower() for i in re.split(r'\W+',title) if i]
        p = [re.sub(r'[^А-я,0-9]', '', i) for i in r]
        p1 = list(filter(lambda i: i, p))
        cur = [word for word in p1 if (pos(word) not in functors_pos and len(word)>1)]
        cur = [u.lower() for u in cur if not u.lower() in list_mest]
        cur = [i for i in cur if not i in russian_stopwords]
        cur = m.lemmatize(' '.join(cur))
#         cur = [morph.parse(o)[0].normal_form for o in cur]
        
        title = ''
        for i in cur:
            if len(i)>2:
                title += i + ' '
#         print(title)
        doc_to_t[doc_id] = title
print (len(doc_to_t))

28026


In [19]:
import pandas as pd
train_d = pd.read_csv('train_groups.csv')
traingroups_titled = {}
for i in range(len(train_d)):
    new_doc = train_d.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    title = doc_to_t[doc_id]
#     title = [u.lower() for u in title if not u.lower() in list_mest]
#     title = [morph.parse(o)[0].normal_form for o in title]
    if doc_group not in traingroups_titled:
        traingroups_titled[doc_group] = []
    traingroups_titled[doc_group].append((doc_id, title, target))

In [ ]:
# %%time
# # вот этот кусок кода!!!!!!!!!!!!!!!!!!!

# import numpy as np
# import Levenshtein
# import numpy as np
# import gensim
# import nltk as nl
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# from gensim.models import Word2Vec
# y_train1 = []
# X_train1 = []
# groups_train1 = []
# a = 0
# b = 1000
# for new_group1 in traingroups_titled:
#     docs1 = traingroups_titled[new_group1]
#     list_data = []
#     ls = []
#     aa = 0
#     for k, (doc_id, title, target_id) in enumerate(docs1):
#         aa+=1
#         title = title.strip().split()
#         title1 = [u.lower() for u in title if not u.lower() in list_mest]
#         if len(title1)==0:
#             for e in range(10):
#                 title1.append(str(k))
#         elif len(title1)<10:
#             for e in range(10-len(title1)):
#                 title1.append(title1[0])
#         ls.append(title1[:10])
#         list_data.append(' '.join(title1))
#     print(aa)
#     print(len(ls))
#     vectorizer = TfidfVectorizer(max_features=20)
#     model = Word2Vec(ls,size = 10)
#     wordss = list(model.wv.vocab)
#     print(wordss)
#     vectorss = []
#     for word in wordss:
#         vectorss.append(model[word].tolist())
#     data11 = np.array(vectorss)
#     data11 = np.sort(data11, axis=0)
#     print(data11.shape)
#     vectorss = []
#     for word in wordss:
#         vectorss.append(model[word].tolist())
#     X6 = vectorizer.fit_transform(list_data)
#     dic = set(vectorizer.get_feature_names())
#     print(dic)
#     vectorizer = CountVectorizer(max_features=20).fit_transform(list_data)
#     vectores = vectorizer.toarray()
# #     dist = cosine_similarity(vectores, vectores)
#     dist = cosine_similarity(data11, data11)
#     print(dist.shape)
#     for k1, (doc_id1, title1, target_id1) in enumerate(docs1):
#         y_train1.append(target_id1)
#         groups_train1.append(new_group1)
#         words1 = set(title1.strip().split())
#         sum1 = 0
#         all_dist2 = []
#         all_dist4 = []
#         all_dist1 = []
#         for j in range(0, len(docs1)):
#             print(j)
#             if k1 == j:
#                 continue
#             doc_id_j, title_j, target_j = docs1[j]
#             words_j = set(title_j.strip().split())
#             y = len(words1.intersection(words_j))
#             y1 = Levenshtein.distance(title1,title_j)
#             y2 = dist[k1,j]
#             all_dist4.append(y2)
#             all_dist2.append(y1)
#             all_dist1.append(y)
#         for i in sorted(all_dist1, reverse=True)[0:15]:
#             sum1+=i
#         sum2 = 0
#         sum4 = 0
#         ss = 0
#         for i in sorted(all_dist2, reverse=False)[0:5]:
#             sum2+=i
#         for i in sorted(all_dist4, reverse=False)[0:15]:
#             sum4+=i
#         for i in sorted(all_dist2, reverse=False)[0:15]:
#             ss+=i
#         all_dist3=len(words1.intersection(dic))
#         X_train1.append(sorted(all_dist4, reverse=False)[0:15]+sorted(all_dist4, reverse=False)[15:2:25]+[all_dist3,sum4,sum1,ss,len(title1.strip().split())] + sorted(all_dist1, \
#                                 reverse=True)[0:5])
# X_train1 = np.array(X_train1)
# y_train1 = np.array(y_train1)
# groups_train1 = np.array(groups_train1)
# print (X_train1.shape, y_train1.shape, groups_train1.shape)

In [20]:
# %%time
# # вот этот кусок кода!!!!!!!!!!!!!!!!!!!

# import numpy as np
# import Levenshtein
# import numpy as np
# import gensim
# import nltk as nl
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# from gensim.models import Word2Vec
# y_train1 = []
# X_train1 = []
# groups_train1 = []
# a = 0
# b = 1000
# for new_group1 in traingroups_titled:
#     docs1 = traingroups_titled[new_group1]
#     list_data = []
#     ls = []
#     aa = 0
#     for k, (doc_id, title, target_id) in enumerate(docs1):
#         title = title.strip().split()
#         title1 = [u.lower() for u in title if not u.lower() in list_mest]
#         list_data.append(' '.join(title1))
#     vectorizer = TfidfVectorizer(max_features=10)
#     X6 = vectorizer.fit_transform(list_data)
#     dic = set(vectorizer.get_feature_names())
#     print(dic)
#     vectorizer = CountVectorizer(max_features=20).fit_transform(list_data)
#     vectores = vectorizer.toarray()
#     dist = cosine_similarity(X6, X6)
#     for k1, (doc_id1, title1, target_id1) in enumerate(docs1):
#         y_train1.append(target_id1)
#         groups_train1.append(new_group1)
#         words1 = set(title1.strip().split())
#         sum1 = 0
#         all_dist4 = []
#         for j in range(0, len(docs1)):
#             if k1 == j:
#                 continue
#             doc_id_j, title_j, target_j = docs1[j]
#             words_j = set(title_j.strip().split())
#             y2 = dist[k1,j]
#             all_dist4.append(y2)
#         sum4 = 0
#         for i in sorted(all_dist4, reverse=False)[0:15]:
#             sum4+=i
#         all_dist3=len(words1.intersection(dic))
#         X_train1.append(sorted(all_dist4, reverse=False)[0:15]+[all_dist3,sum4,len(title1.strip().split())])
# X_train1 = np.array(X_train1)
# y_train1 = np.array(y_train1)
# groups_train1 = np.array(groups_train1)
# print (X_train1.shape, y_train1.shape, groups_train1.shape)

In [31]:
scaler = StandardScaler()
scaler.fit(X_train4)
X_train4 = scaler.transform(X_train4)

In [32]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn import linear_model
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression

max = []
where = 0
mmm = 0
for g in range(20):
    kf = KFold(n_splits=3)
    score = np.arange(60,dtype = float).reshape(3,-1)
    a = 0
    for train,test in kf.split(X_train4):

        X_tr = X_train4[train]
        y_tr = y_train1[train]
        X_t = X_train4[test]
        y_t = y_train1[test]
#         clf = LogisticRegression(random_state=0)
        clf = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=8,
                      alpha = 0.01, C=100,batch_size = 100)
        n_estimators = 1200
        learning_rate = 0.3
        max_depth = 6
#         clf2 = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
#                         max_depth=max_depth, nthread=4)
#         clf2.fit(X_tr, y_tr)
#         pr = clf2.predict(X_t)
        clf.fit(X_tr, y_tr)
        my_clf_pred = clf.predict(X_t)
        
        
        for i in range(20,40):
    #         pred = (np.array(my_clf_pred>0.01*i,dtype=int)) 
            pred = my_clf_pred
            score[a][i-20] = f1_score(pred>0.01*(i),y_t)
        a+=1
    #     break
    if g==0:
        sc = score.mean(axis = 0)
    else:
        sc += score.mean(axis = 0)
    print(g)
    print(score.mean(axis = 0).max())
#     print(g,"score",sc.max())
#     max = sc.max()
#     print(np.where(sc==max)[0][0])
#     where +=np.where(sc==max)[0][0]
# #     print(sc)
# #     print(max,'max')
#     mmm+=sc.max()
# print('mean',mmm/20)
# print('mean',where/20)
print(sc.max()/20)
print(np.where(sc==sc.max())[0][0]+20)

/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


0
0.6915205156874199
1
0.6987793850031934
2
0.6980142276798483
3
0.6932894401347299
4
0.6932077591274332
5
0.6974114657426947
6
0.6991348676448134
7
0.694225341160618
8
0.6965404127858174
9
0.6979606873531256
10
0.6993303499469136
11
0.7001221089424464
12
0.6978864420977904
13
0.7014415327015224
14
0.6977401467206547
15
0.690906810712668
16
0.5865179708047383
17
0.6952816934633672
18
0.6794777484876923
19
0.7012879800406573
0.6887549400423548
29


In [52]:
X_train111 = np.hstack((X_train3, X_train4[:,0:5]))

In [53]:
scaler = StandardScaler()
scaler.fit(X_train111)
X_train111 = scaler.transform(X_train111)

In [54]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn import linear_model
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression

max = []
where = 0
mmm = 0
for g in range(20):
    kf = KFold(n_splits=3)
    score = np.arange(60,dtype = float).reshape(3,-1)
    a = 0
    for train,test in kf.split(X_train111):

        X_tr = X_train111[train]
        y_tr = y_train1[train]
        X_t = X_train111[test]
        y_t = y_train1[test]
#         clf = LogisticRegression(random_state=0)
        clf = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=8,
                      alpha = 0.01, C=100,batch_size = 100)
        n_estimators = 1200
        learning_rate = 0.3
        max_depth = 6
#         clf2 = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
#                         max_depth=max_depth, nthread=4)
#         clf2.fit(X_tr, y_tr)
#         pr = clf2.predict(X_t)
        clf.fit(X_tr, y_tr)
        my_clf_pred = clf.predict(X_t)
        
        
        for i in range(20,40):
    #         pred = (np.array(my_clf_pred>0.01*i,dtype=int)) 
            pred = my_clf_pred
            score[a][i-20] = f1_score(pred>0.01*(i),y_t)
        a+=1
    #     break
    if g==0:
        sc = score.mean(axis = 0)
    else:
        sc += score.mean(axis = 0)
    print(g)
    print(score.mean(axis = 0).max())
#     print(g,"score",sc.max())
#     max = sc.max()
#     print(np.where(sc==max)[0][0])
#     where +=np.where(sc==max)[0][0]
# #     print(sc)
# #     print(max,'max')
#     mmm+=sc.max()
# print('mean',mmm/20)
# print('mean',where/20)
print(sc.max()/20)
print(np.where(sc==sc.max())[0][0]+20)

/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


0
0.7324774940598419
1
0.7186241860969437
2
0.7031037807884836
3
0.7307294641152042
4
0.7261866892406431


KeyboardInterrupt: 

In [21]:
%%time
# вот этот кусок кода!!!!!!!!!!!!!!!!!!!

import numpy as np
import Levenshtein
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
y_train1 = []
X_train3 = []
groups_train1 = []
for new_group1 in traingroups_titled:
    docs1 = traingroups_titled[new_group1]
    list_data = []
    for k, (doc_id, title, target_id) in enumerate(docs1):
        title = title.strip().split()
        title1 = [u.lower() for u in title if not u.lower() in list_mest]
        list_data.append(' '.join(title1))
    vectorizer = TfidfVectorizer(max_features=20)
    X6 = vectorizer.fit_transform(list_data)
    dic = set(vectorizer.get_feature_names())
#     print(dic)
    vectorizer = CountVectorizer(max_features=20).fit_transform(list_data)
    vectores = vectorizer.toarray()
    dist = cosine_similarity(X6, X6)
    for k1, (doc_id1, title1, target_id1) in enumerate(docs1):
        y_train1.append(target_id1)
        groups_train1.append(new_group1)
        words1 = set(title1.strip().split())
        sum1 = 0
        all_dist2 = []
        all_dist4 = []
        all_dist1 = []
        for j in range(0, len(docs1)):
            if k1 == j:
                continue
            doc_id_j, title_j, target_j = docs1[j]
            words_j = set(title_j.strip().split())
            y = 0
            if (len(words1)+len(words_j)-len(words1.intersection(words_j)))!=0:
                y = len(words1.intersection(words_j))/(len(words1)+len(words_j)-len(words1.intersection(words_j)))
#             y = len(words1.intersection(words_j))
            y1 = Levenshtein.distance(title1,title_j)
            y2 = dist[k1,j]
            all_dist4.append(y2)
            all_dist2.append(y1)
            all_dist1.append(y)
        for i in sorted(all_dist1, reverse=True)[0:15]:
            sum1+=i
        sum2 = 0
        sum4 = 0
        ss = 0
        for i in sorted(all_dist2, reverse=False)[0:5]:
            sum2+=i
        for i in sorted(all_dist4, reverse=False)[0:15]:
            sum4+=i
        for i in sorted(all_dist2, reverse=False)[0:15]:
            ss+=i
        all_dist3=len(words1.intersection(dic))
        f = 'купить' in words1
#         if f==1:
#             print(1)
        X_train3.append(sorted(all_dist4, reverse=False)[0:15]+sorted(all_dist4, reverse=False)[15:2:25]+[all_dist3,sum4,sum1,ss,len(title1.strip().split()),f] + sorted(all_dist1, \
                                reverse=True)[0:5])
X_train3 = np.array(X_train3)
y_train1 = np.array(y_train1)
groups_train1 = np.array(groups_train1)
print (X_train3.shape, y_train1.shape, groups_train1.shape)

(11690, 26) (11690,) (11690,)
CPU times: user 8.29 s, sys: 11.8 ms, total: 8.3 s
Wall time: 8.31 s


In [13]:
params = product(
    (400, 600, 800, 1000, 1200), # n_estimators
    (0.01, 0.1, 0.2, 0.3), # learning_rate
    (3, 6, 9), # max_depth
)
res_score = 0

for n_estimators, learning_rate, max_depth in list(params):
    clf = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
                        max_depth=max_depth, nthread=4)
#     clf = XGBClassifier(silent=False, 
#                       scale_pos_weight=1,
#                       learning_rate=0.01,  
#                       colsample_bytree = 0.4,
#                       subsample = 0.8,
#                       objective='binary:logistic', 
#                       n_estimators=1000, 
#                       reg_alpha = 0.3,
#                       max_depth=9, 
#                       gamma=10)
    res = cross_validate(clf, X_train3, y_train1, cv=3, scoring='f1')
    score = res['test_score'].mean()
    print(score)
    print(clf)
    if score > res_score:
        res_score = score
        res_model = clf
        
res_model, res_score

0.6975849463107835
XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=0.01, max_delta_step=None, max_depth=3,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=400, n_jobs=None, nthread=4, num_parallel_tree=None,
              objective='binary:logistic', random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, validate_parameters=None, verbosity=None)
0.6940893836286416
XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=0.01, max_delta_step=None, max_depth=6,
   

0.7017296078589622
XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=0.01, max_delta_step=None, max_depth=3,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=600, n_jobs=None, nthread=4, num_parallel_tree=None,
              objective='binary:logistic', random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, validate_parameters=None, verbosity=None)
0.6955151587801628
XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=0.01, max_delta_step=None, max_depth=6,
   

0.7015098224178957
XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=0.01, max_delta_step=None, max_depth=3,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=800, n_jobs=None, nthread=4, num_parallel_tree=None,
              objective='binary:logistic', random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, validate_parameters=None, verbosity=None)
0.6983616274832697
XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=0.01, max_delta_step=None, max_depth=6,
   

0.7022131799542438
XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=0.01, max_delta_step=None, max_depth=3,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=1000, n_jobs=None, nthread=4, num_parallel_tree=None,
              objective='binary:logistic', random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, validate_parameters=None, verbosity=None)
0.6982086406799112
XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=0.01, max_delta_step=None, max_depth=6,
  

0.7011106691624032
XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=0.01, max_delta_step=None, max_depth=3,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=1200, n_jobs=None, nthread=4, num_parallel_tree=None,
              objective='binary:logistic', random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, validate_parameters=None, verbosity=None)
0.6993556826781543
XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=0.01, max_delta_step=None, max_depth=6,
  

(XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
               colsample_bynode=None, colsample_bytree=None, gamma=None,
               gpu_id=None, importance_type='gain', interaction_constraints=None,
               learning_rate=0.01, max_delta_step=None, max_depth=3,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               n_estimators=1000, n_jobs=None, nthread=4, num_parallel_tree=None,
               objective='binary:logistic', random_state=None, reg_alpha=None,
               reg_lambda=None, scale_pos_weight=None, subsample=None,
               tree_method=None, validate_parameters=None, verbosity=None),
 0.7022131799542438)

In [123]:
from sklearn.svm import SVC

params = product(
    (1.0,5.0,10.0,100.0), # n_estimators
    ('linear', 'poly', 'rbf', 'sigmoid'), # learning_rate
    ('scale', 'auto'), # max_depth
)
# C=1.0, kernel='rbf', degree=3, gamma='scale', coef0=0.0, shrinking=True, 
res_score = 0

for a, b, c1 in list(params):
#     clf = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
#                         max_depth=max_depth, nthread=4)
    clf = SVC(gamma=c1,C = a,kernel=b)
#     clf = XGBClassifier(silent=False, 
#                       scale_pos_weight=1,
#                       learning_rate=0.01,  
#                       colsample_bytree = 0.4,
#                       subsample = 0.8,
#                       objective='binary:logistic', 
#                       n_estimators=1000, 
#                       reg_alpha = 0.3,
#                       max_depth=9, 
#                       gamma=10)
    res = cross_validate(clf, X_train3, y_train1, cv=3, scoring='f1')
    score = res['test_score'].mean()
    print(score)
    print(clf)
    if score > res_score:
        res_score = score
        res_model = clf
        
res_model, res_score

0.62816837055434
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
0.62816837055434
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
0.49688309658055
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
0.4982512155470891
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
    max_iter=-1, probabil

0.6183532649369249
SVC(C=100.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='sigmoid',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


(SVC(C=10.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
     max_iter=-1, probability=False, random_state=None, shrinking=True,
     tol=0.001, verbose=False), 0.6715781676519326)

In [61]:
import numpy as np
y_train = []
X_train = []
groups_train = []
for new_group in traingroups_titled:
    docs = traingroups_titled[new_group]
    for k, (doc_id, title, target_id) in enumerate(docs):
        y_train.append(target_id)
        groups_train.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        sum1 = 0
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, target_j = docs[j]
            words_j = set(title_j.strip().split())
            y = len(words.intersection(words_j))
            all_dist.append(y)
            sum1+=y
        X_train.append(sorted(all_dist, reverse=True)[0:15] +[sum1,len(title.strip().split())]   )
X_train = np.array(X_train)
y_train = np.array(y_train)
groups_train = np.array(groups_train)
print (X_train.shape, y_train.shape, groups_train.shape)

(11690, 17) (11690,) (11690,)


In [36]:
scaler = StandardScaler()
scaler.fit(X_train3)
X_train3 = scaler.transform(X_train3)

In [39]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn import linear_model
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression

max = []
where = 0
mmm = 0
for g in range(20):
    kf = KFold(n_splits=3)
    score = np.arange(60,dtype = float).reshape(3,-1)
    a = 0
    for train,test in kf.split(X_train3):

        X_tr = X_train3[train]
        y_tr = y_train1[train]
        X_t = X_train3[test]
        y_t = y_train1[test]
#         clf = LogisticRegression(random_state=0)
        clf = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=6,
                      alpha = 0.01, C=50,batch_size = 100)
        n_estimators = 1200
        learning_rate = 0.3
        max_depth = 6
#         clf2 = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
#                         max_depth=max_depth, nthread=4)
#         clf2.fit(X_tr, y_tr)
#         pr = clf2.predict(X_t)
        clf.fit(X_tr, y_tr)
        my_clf_pred = clf.predict(X_t)
        
        
        for i in range(20,40):
    #         pred = (np.array(my_clf_pred>0.01*i,dtype=int)) 
            pred = my_clf_pred
            score[a][i-20] = f1_score(pred>0.01*(i),y_t)
        a+=1
    #     break
    if g==0:
        sc = score.mean(axis = 0)
    else:
        sc += score.mean(axis = 0)
    print(g)
    print(score.mean(axis = 0).max())
#     print(g,"score",sc.max())
#     max = sc.max()
#     print(np.where(sc==max)[0][0])
#     where +=np.where(sc==max)[0][0]
# #     print(sc)
# #     print(max,'max')
#     mmm+=sc.max()
# print('mean',mmm/20)
# print('mean',where/20)
print(sc.max()/20)
print(np.where(sc==sc.max())[0][0]+20)

/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


0
0.739184555711503
1
0.7358582230743552
2
0.7275984208631096
3
0.7169384074152924
4
0.7366475113773037
5
0.7320368372977777
6
0.7134577032387398
7
0.7275604714415232
8
0.7340347819727112
9
0.7356230149545394
10
0.7302590038692575
11
0.7257841567171898
12
0.7333976892224254
13
0.7362622579083192
14
0.7359377742805888
15
0.7300881483519647
16
0.6826871661502039
17
0.7294699632367013
18
0.7338260738647406
19
0.7263242727690441
0.7267851993712298
27


In [43]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score
kf = KFold(n_splits=3)
score = 0
c_b = 0
a_b = 0
me_b = 0
m_b = ""
C = [10,50,100,500]
alpha = [0.01]
max_epoch = [5,6,7,8]
model_type = ["log_reg"]
for i in C:
    for j in alpha:
        for k in max_epoch:
            for z in model_type:
                a = 0
                for train,test in kf.split(X_train3):
                        X_tr = X_train3[train]
                        y_tr = y_train1[train]
                        X_t = X_train3[test]
                        y_t = y_train1[test]
                        clf = MySGDClassifier(batch_generator, model_type=z, max_epoch=k,
                                      alpha = j, C=i,batch_size = 100)
                        clf.fit(X_tr, y_tr)
                        my_clf_pred = clf.predict(X_t)
                        a = a+roc_auc_score(y_t,my_clf_pred)
                print(a/3,i,j,k,z)
                if(a/3>score):
                    score = a/3
                    c_b = i
                    a_b = j
                    me_b = k
                    m_b = z
print(score,"score")
print(c_b,"C")
print(a_b,"alpha")
print(me_b,"epoch")
print(m_b,"model_type")

/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


0.8424045144185196 10 0.01 5 log_reg


/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


0.8927299321672016 10 0.01 6 log_reg


/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


...

0.892692628081992 500 0.01 6 log_reg


/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


0.8914026166492985 500 0.01 7 log_reg


/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


0.8931749864771401 500 0.01 8 log_reg
0.8936882748798869 score
100 C
0.01 alpha
7 epoch
log_reg model_type


In [52]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
max = []
where = 0
mmm = 0
for g in range(20):
    kf = KFold(n_splits=3)
    score = np.arange(60,dtype = float).reshape(3,-1)
    a = 0
    for train,test in kf.split(X_train):

        X_tr = X_train[train]
        y_tr = y_train1[train]
        X_t = X_train[test]
        y_t = y_train1[test]
        X_tr1 = X_train1[train]
        X_t1 = X_train1[test]
#         clf = LogisticRegression(random_state=0)
        clf = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=6,
                      alpha = 0.01, C=50,batch_size = 100)
        clf.fit(X_tr, y_tr)
        my_clf_pred = clf.predict(X_t)
        res1 = (my_clf_pred>0.01*(26)).reshape(-1,1)
        
        
        
        
        n_estimators = 1200
        learning_rate = 0.001
        max_depth = 6
        clf2 = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
                        max_depth=max_depth, nthread=4)
        clf2.fit(X_tr, y_tr)
        pr = clf2.predict(X_t).reshape(-1,1)
        clf1 = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=6,
                      alpha = 0.01, C=50,batch_size = 100)
        clf1.fit(X_tr1, y_tr)
        my_clf_pred1 = clf1.predict(X_t1)
        res2 = (my_clf_pred1>0.01*(28)).reshape(-1,1)
        clf3 = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
                        max_depth=max_depth, nthread=4)
        clf3.fit(X_tr1, y_tr)
        pr1 = clf3.predict(X_t1).reshape(-1,1)
        
        X_res = np.concatenate((res1,pr,res2,pr1), axis=1)
        y_res =  (np.mean(X_res, axis = 1))>0.5
        print(X_res.shape)
        print(y_res.shape)
        print(y_t.shape)
        for i in range(20,40):
    #         pred = (np.array(my_clf_pred>0.01*i,dtype=int)) 
            pred = my_clf_pred
            score[a][i-20] = f1_score(y_res,y_t)
        a+=1
    #     break
    if g==0:
        sc = score.mean(axis = 0)
    else:
        sc += score.mean(axis = 0)
    print(g)
    print(score.mean(axis = 0).max())
#     print(g,"score",sc.max())
#     max = sc.max()
#     print(np.where(sc==max)[0][0])
#     where +=np.where(sc==max)[0][0]
# #     print(sc)
# #     print(max,'max')
#     mmm+=sc.max()
# print('mean',mmm/20)
# print('mean',where/20)
print(sc.max()/20)
print(np.where(sc==sc.max())[0][0]+20)

/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


(3897, 4)
(3897,)
(3897,)


KeyboardInterrupt: 

In [38]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
max = []
where = 0
mmm = 0
www = np.zeros(2)
for g in range(5):
    kf = KFold(n_splits=3)
    score = np.arange(60,dtype = float).reshape(3,-1)
    a = 0
    for train,test in kf.split(X_train3):

        X_tr = X_train3[train]
        y_tr = y_train1[train]
        X_t = X_train3[test]
        y_t = y_train1[test]
        X_tr1 = X_train4[train]
        X_t1 = X_train4[test]
        clf = LogisticRegression(random_state=0)
        clf = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=5,
                      alpha = 0.01, C=100,batch_size = 100)
        clf.fit(X_tr, y_tr)
        my_clf_pred = clf.predict(X_t).reshape(-1,1)
        clf1 = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=6,
                      alpha = 0.01, C=50,batch_size = 100)
        clf1.fit(X_tr1, y_tr)
        my_clf_pred1 = clf1.predict(X_t1).reshape(-1,1)
#         res2 = (my_clf_pred1>0.01*(29)).reshape(-1,1)
#         clf3 = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
#                         max_depth=max_depth, nthread=4)
#         clf3.fit(X_tr1, y_tr)
#         pr1 = clf3.predict(X_t1).reshape(-1,1)
        
        X_res = np.concatenate((my_clf_pred1,my_clf_pred), axis=1)
#         y_res =  (np.mean(X_res, axis = 1))>0.5
        print(X_res.shape)
#         print(y_res.shape)
        print(y_t.shape)
        clf5 = MySGDClassifier(batch_generator, model_type="lin_reg", max_epoch=6,
                      alpha = 0.01, C=50,batch_size = 100)
        reg = LinearRegression().fit(X_res, y_t)
        www+=reg.coef_
        print(www/(g*3+3))
#     print(g,"score",sc.max())
#     max = sc.max()
#     print(np.where(sc==max)[0][0])
#     where +=np.where(sc==max)[0][0]
# #     print(sc)
# #     print(max,'max')
#     mmm+=sc.max()
# print('mean',mmm/20)
# print('mean',where/20)
print(www/10)

/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


(3897, 2)
(3897,)
[0.22731289 0.19629139]


/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


(3897, 2)
(3897,)
[0.32913224 0.45895287]


/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


...

(3897, 2)
(3897,)
[0.28928379 0.47072833]


/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


(3897, 2)
(3897,)
[0.34586784 0.61278873]


/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


KeyboardInterrupt: 

In [38]:
www/30

array([0.22154872, 0.14664111, 0.14601091, 0.20563443])

In [65]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
max = []
where = 0
mmm = 0
a1 = 0
for g in range(20):
    kf = KFold(n_splits=3)
    score = np.arange(300,dtype = float).reshape(3,-1)
    a = 0
    for train,test in kf.split(X_train3):

        X_tr = X_train3[train]
        y_tr = y_train1[train]
        X_t = X_train3[test]
        y_t = y_train1[test]
        X_tr1 = X_train4[train]
        X_t1 = X_train4[test]
#         clf = LogisticRegression(random_state=0)
        clf = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=5,
                      alpha = 0.01, C=100,batch_size = 100)
        clf.fit(X_tr, y_tr)
        my_clf_pred = clf.predict(X_t).reshape(-1,1)
#         res1 = (my_clf_pred>0.01*(29))
        
        
        
        
#         n_estimators = 1000
#         learning_rate = 0.001
#         max_depth = 3
#         clf2 = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
#                         max_depth=max_depth, nthread=4)
#         clf2.fit(X_tr, y_tr)
#         pr = clf2.predict(X_t).reshape(-1,1)
        clf1 = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=6,
                      alpha = 0.01, C=50,batch_size = 100)
        clf1.fit(X_tr1, y_tr)
        my_clf_pred1 = clf1.predict(X_t1).reshape(-1,1)
#         res2 = (my_clf_pred1>0.01*(29)).reshape(-1,1)
#         clf3 = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
#                         max_depth=max_depth, nthread=4)
#         clf3.fit(X_tr1, y_tr)
#         pr1 = clf3.predict(X_t1).reshape(-1,1)
#         print(res1[:20])
#         print(pr[:20])
#         X_res = np.concatenate((my_clf_pred1,my_clf_pred), axis=1)
#         print(my_clf_pred1)
        y_res =  (0.54937047*my_clf_pred1+my_clf_pred* 1.05119227)/(0.54937047+1.05119227)
#         print(y_res)
#         y_res =  (res1+pr* 0.0)
#         print(y_res[:10])
#         print(X_res.shape)
#         print(y_res.shape)
#         print(y_t.shape)
        for i in range(0,100):
    #         pred = (np.array(my_clf_pred>0.01*i,dtype=int)) 
#             pred = my_clf_pred
            score[a][i] = f1_score(y_res>0.01*27,y_t)
        a+=1
    #     break
#     print(score)
    if g==0:
        sc = score.mean(axis = 0)
    else:
        sc += score.mean(axis = 0)
    print(g)
    print(score.mean(axis = 0).max())
#     print(g,"score",sc.max())
#     max = sc.max()
    print(np.where(score.mean(axis = 0)==(score.mean(axis = 0)).max())[0][0])
    a2=np.where(score.mean(axis = 0)==(score.mean(axis = 0)).max())[0][0]
    a1+=a2
    print(a2)
#     where +=np.where(sc==max)[0][0]
# #     print(sc)
# #     print(max,'max')
#     mmm+=sc.max()
# print('mean',mmm/20)
# print('mean',where/20)
print(sc.max()/20)
print(np.where(sc==sc.max())[0][0])
a1/20

/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


0
0.737043355790663
0
0
1
0.7367933386779232
0
0
2
0.7343059919801499
0
0
3
0.7329142128542362
0
0
4
0.7363481748033948
0
0
5
0.7328052228415247
0
0
6
0.7354397747305511
0
0
7
0.7332770832748672
0
0


KeyboardInterrupt: 

In [55]:
test_data1 = pd.read_csv('test_groups.csv')
test_groups_titled1 = {}
for i in range(len(test_data1)):
    new_doc1 = test_data1.iloc[i]
    doc_group1 = new_doc1['group_id']
    doc_id1 = new_doc1['doc_id']
    title1 = doc_to_t[doc_id1]
    
#     title1 = [u.lower() for u in title1 if not u.lower() in list_mest]
#     title1 = [morph.parse(o)[0].normal_form for o in title1]
    if doc_group1 not in test_groups_titled1:
        test_groups_titled1[doc_group1] = []
    test_groups_titled1[doc_group1].append((doc_id1, title1))

In [56]:
%%time
X_test1 = []
groups_test1 = []
p1 = 0
for new_group in test_groups_titled1:
    p1+=1
    print(p1)
    docs1 = test_groups_titled1[new_group]
    list_data = []
    for k, (doc_id, title) in enumerate(docs1):
        title = title.strip().split()
        title1 = [u.lower() for u in title if not u.lower() in list_mest]
        list_data.append(' '.join(title1))
    vectorizer = TfidfVectorizer(max_features=20)
    X6 = vectorizer.fit_transform(list_data)
    dic = set(vectorizer.get_feature_names())
    print(dic)
    vectorizer = CountVectorizer(max_features=20).fit_transform(list_data)
    vectores = vectorizer.toarray()
    dist = cosine_similarity(X6, X6)
    for k, (doc_id, title) in enumerate(docs1):
        groups_test1.append(new_group)
        all_dist4 = []
        all_dist2 = []
        words = set(title.strip().split())
        sum1 = 0
#         dist = np.zeros((len(docs1), len(docs1)))
        all_dist1 = []
        n = 0
        for j in range(0, len(docs1)):
            if k == j:
                continue
            n+=1
            doc_id_j, title_j= docs1[j]
            words_j = set(title_j.strip().split())
            y = 0
            if (len(words)+len(words_j)-len(words.intersection(words_j)))!=0:
                y = len(words.intersection(words_j))/(len(words)+len(words_j)-len(words.intersection(words_j)))
#             y = len(words1.intersection(words_j))
            y1 = Levenshtein.distance(title,title_j)
            all_dist2.append(y1)
            y2 = dist[k,j]
#             print(y2)
            all_dist4.append(y2)
            all_dist1.append(y)
#             sum1+=y
        for i in sorted(all_dist1, reverse=True)[0:15]:
            sum1+=i
        sum2 = 0
        for i in sorted(all_dist2, reverse=False)[0:5]:
            sum2+=i
        sum4 = 0
        for i in sorted(all_dist4, reverse=False)[0:15]:
            sum4+=i
        ss = 0
        for i in sorted(all_dist2, reverse=False)[0:15]:
            ss+=i
        all_dist3=len(words.intersection(dic))
        f = 'купить' in words
#         X_train3.append(sorted(all_dist4, reverse=False)[0:15]+sorted(all_dist4, reverse=False)[15:2:25]+[all_dist3,sum4,sum1,ss,len(title1.strip().split()),f] + sorted(all_dist1, \
#                                 reverse=True)[0:5])
        X_test1.append(sorted(all_dist4, reverse=False)[0:15] +sorted(all_dist4, reverse=False)[15:2:25]+[all_dist3, sum4,sum1,ss,len(title.strip().split()),f] +sorted(all_dist1, reverse=True)[0:5]  )
#         print(sum2)
X_test1 = np.array(X_test1)
groups_test1 = np.array(groups_test1)
print (X_test1.shape, groups_test1.shape)

1
{'гта', 'сервер', 'видео', 'давать', 'сайт', 'интернет', 'самп', 'помощь', 'админ', 'прописывать', 'ксс', 'страница', 'скачать', 'плагин', 'создавать', 'админка', 'статья', 'форум', 'игра', 'блог'}
2
{'центр', 'отзыв', 'крокус', 'красногорск', 'работа', 'россия', 'группа', 'москва', 'торговый', 'контакт', 'сити', 'метро', 'международный', 'добираться', 'строгино', 'компания', 'магазин', 'мякинино', 'вакансия', 'адрес'}
3
{'завтра', 'год', 'балашов', 'дневник', 'день', 'самый', '2016', 'точный', 'прогноз', 'россия', 'балашовский', 'саратовский', 'новость', 'мир', 'район', 'почасовой', 'русский', 'область', 'тема', 'погода'}
4
{'магия', 'растение', 'бесплатно', 'лечить', 'книга', 'борщевик', 'ожог', 'новость', 'онлайн', 'мир', 'страница', 'скачать', 'лечение', 'сервис', 'форум', 'обсуждение', 'тема', 'дневник', 'ядовитый', 'российский'}
5
{'заготовка', 'вконтакте', 'ягода', 'смородина', 'домашний', 'онлайн', 'протертый', 'фото', 'лечение', 'рецепт', 'сервис', 'зима', 'форум', 'обсужден

40
{'солнцезащитный', 'очки', 'выбирать', 'женский', 'отзыв', 'интернет', 'линза', 'зрение', 'медицинский', 'оптика', 'москва', 'контактный', 'бифокальный', 'телефон', 'салон', 'цена', 'лечение', 'коррекция', 'дневник', 'глаз'}
41
{'бесплатно', 'применение', 'комаровский', 'отзыв', 'показание', 'инструкция', 'ребенок', 'врач', 'миопатия', 'лекарство', 'страница', 'скачать', 'совет', 'лечение', 'помогать', 'дибазол', 'форум', 'средство', 'педиатрия', 'инфекционный'}
42
{'год', 'образование', 'деятельность', 'средний', 'бюджетный', 'сайт', 'профессиональный', 'комиссия', 'государственный', 'университет', '2010', 'новость', 'колледж', 'страница', 'педагогический', 'научно', 'вуз', 'учреждение', 'область', 'образовательный'}
43
{'роутер', 'купить', 'компьютер', 'фай', 'сменить', 'пароль', 'поставлять', 'китайский', 'дешево', 'телефон', 'узнавать', 'модем', 'забывать', 'посмотреть', 'сеть', 'поменять', '300', 'настройка', 'настраивать', 'ответ'}
44
{'год', 'социальный', 'деятельность', 'эко

77
{'растение', 'сохнуть', 'фикус', 'цветок', 'хост', 'вконтакте', 'уход', 'делать', 'онлайн', 'хоста', 'комнатный', 'желтеть', 'болезнь', 'сервис', 'почему', 'лист', 'форум', 'дневник', 'сад', 'российский'}
78
{'бесплатно', 'история', 'родосский', 'колосс', 'культура', 'книга', 'онлайн', 'читать', 'цивилизация', 'искусство', 'скачать', 'библиотека', 'греция', 'архив', 'свет', 'форум', 'игра', 'часть', 'дневник', 'древний'}
79
{'повар', 'фольга', 'приготовление', 'осетр', 'духовка', 'готовить', 'запекать', 'пошаговый', 'кулинарный', 'вкусный', 'целиком', 'фото', 'блюдо', 'рыба', 'приготавливать', 'стерлядь', 'целый', 'рецепт', 'русский', 'ответ'}
80
{'бесплатно', 'история', 'глаголать', 'авторский', 'вопрос', 'уголовный', 'онлайн', 'зарегистрировать', 'мир', 'страница', 'международный', 'регистрация', 'форум', 'юрист', 'гражданин', 'дело', 'право', 'дневник', 'тема', 'архив'}
81
{'бесплатно', 'навык', 'обзор', 'дополнение', 'код', 'горячий', 'грудь', 'чита', 'чит', 'симс', 'увеличивать

116
{'год', 'система', 'доход', 'прибыль', 'проводка', 'работа', 'россия', 'расход', 'дипломный', 'суд', 'новость', 'пример', 'налог', 'бухгалтерский', 'учет', 'форум', 'бухгалтерия', 'продажа', 'госпошлина', 'научный'}
117
{'обзор', 'отзыв', 'инструкция', 'купить', 'техника', 'интернет', 'холодильник', 'стенка', 'лед', 'санкт', 'доставка', 'задний', 'намерзать', 'страница', 'цена', 'совет', 'петербург', 'почему', 'характеристика', 'магазин'}
118
{'год', 'бухгалтер', 'сотрудник', 'ооо', 'вопрос', 'оклад', 'труд', 'работа', 'производственный', 'работник', 'организация', 'оплата', 'зарплата', 'страница', 'учет', 'повышение', 'форум', 'бухгалтерия', 'архив', 'ответ'}
119
{'клиент', 'бесплатно', 'номер', 'мтс', 'оператор', 'тариф', 'частный', 'мобильный', 'интернет', 'телевидение', 'дозваниваться', 'позвонить', 'москва', 'телефон', 'связь', 'услуга', 'сотовый', 'подмосковье', 'область', 'ответ'}
120
{'воротник', 'выбирать', 'женский', 'стильный', 'модный', 'белый', 'модель', 'пиджак', 'нос

153
{'занятый', 'бесплатно', 'спасать', 'серия', 'сайт', 'книга', 'аниме', 'проза', 'делать', 'время', 'онлайн', 'смотреть', 'страница', 'скачать', 'библиотека', 'мочь', 'форум', 'русский', 'апокалипсис', 'архив'}
154
{'бесплатно', 'вода', 'порча', 'заговор', 'книга', 'запись', 'сглазить', 'онлайн', 'скачать', 'определять', 'библиотека', 'узнавать', 'снимать', 'снятие', 'сервис', 'форум', 'сглаз', 'обсуждение', 'дневник', 'российский'}
155
{'видео', 'версия', 'обзор', 'купить', '145', 'устройство', 'прошивка', 'мобильный', 'планшет', 'интернет', 'мобигура', 'смартфон', '450', 'дексп', 'дешево', 'телефон', 'цена', '350', 'магазин', '150'}
156
{'сервер', 'бесплатно', 'тренировка', 'игровой', 'хороший', 'патч', 'ссылка', 'карта', 'контры', 'бот', 'торрент', 'модель', 'загрузка', 'мониторинг', 'скачать', 'страйк', 'плагин', 'устанавливать', 'игра', 'мода'}
157
{'год', 'консультация', 'ооо', 'вопрос', 'государственный', 'директор', 'правовой', 'учредительный', 'суд', 'предприятие', 'нужный'

In [57]:
test_data1 = pd.read_csv('test_groups.csv')
test_groups_titled1 = {}
for i in range(len(test_data1)):
    new_doc1 = test_data1.iloc[i]
    doc_group1 = new_doc1['group_id']
    doc_id1 = new_doc1['doc_id']
    title1 = doc_to_h1[doc_id1]
    
#     title1 = [u.lower() for u in title1 if not u.lower() in list_mest]
#     title1 = [morph.parse(o)[0].normal_form for o in title1]
    if doc_group1 not in test_groups_titled1:
        test_groups_titled1[doc_group1] = []
    test_groups_titled1[doc_group1].append((doc_id1, title1))

In [58]:
%%time
# вот этот кусок кода!!!!!!!!!!!!!!!!!!!

import numpy as np
import Levenshtein
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
X_test4 = []
groups_test1 = []
for new_group1 in test_groups_titled1:
    docs1 = test_groups_titled1[new_group1]
    list_data = []
    for k, (doc_id, title) in enumerate(docs1):
        title = title.strip().split()
        title1 = [u.lower() for u in title if not u.lower() in list_mest]
        list_data.append(' '.join(title1[:20]))
    vectorizer = TfidfVectorizer(max_features=20)
    X6 = vectorizer.fit_transform(list_data)
    dic = set(vectorizer.get_feature_names())
    print(dic)
    vectorizer = CountVectorizer(max_features=20).fit_transform(list_data)
    vectores = vectorizer.toarray()
    dist = cosine_similarity(X6, X6)
    for k1, (doc_id1, title1) in enumerate(docs1):
        groups_test1.append(new_group1)
        words1 = set((title1[:20]).strip().split())
        sum1 = 0
        all_dist2 = []
        all_dist4 = []
        all_dist1 = []
        for j in range(0, len(docs1)):
            if k1 == j:
                continue
            doc_id_j, title_j = docs1[j]
            words_j = set((title_j[:20]).strip().split())
            y = 0
            if (len(words1)+len(words_j)-len(words1.intersection(words_j)))!=0:
                y = len(words1.intersection(words_j))/(len(words1)+len(words_j)-len(words1.intersection(words_j)))
#             y = len(words1.intersection(words_j))
            y1 = Levenshtein.distance(title1[:20],title_j[:20])
            y2 = dist[k1,j]
            all_dist4.append(y2)
            all_dist2.append(y1)
            all_dist1.append(y)
        for i in sorted(all_dist1, reverse=True)[0:15]:
            sum1+=i
        sum2 = 0
        sum4 = 0
        ss = 0
        for i in sorted(all_dist2, reverse=False)[0:5]:
            sum2+=i
        for i in sorted(all_dist4, reverse=False)[0:15]:
            sum4+=i
        for i in sorted(all_dist2, reverse=False)[0:15]:
            ss+=i
        all_dist3=len(words1.intersection(dic))
#         f = 'купить' in words1
#         if f==1:
#             print(1)
        f = ('купить' in words1) or ('продать'in words1)
        if f==1:
            print(1)
        X_test4.append(sorted(all_dist4, reverse=False)[0:2:10]+[all_dist3,sum4,sum1,ss,len(title1.strip().split()),f] + sorted(all_dist1, \
                                reverse=True)[0:15])
#         X_test4.append(sorted(all_dist4, reverse=False)[0:15]+[all_dist3,sum4,sum1,ss,len(title1.strip().split()),f] + sorted(all_dist1, \
#                                 reverse=True)[0:15])
X_test4 = np.array(X_test4)
print (X_test4.shape,len(groups_test1))

{'сервер', 'главный', 'язык', 'закрывать', 'просмотр', 'сайт', 'прописывать', 'запрашивать', 'переходить', 'страница', 'загружать', 'попробовать', 'скачать', 'предыдущий', 'регистрация', 'админка', 'форум', 'контент', 'игра', 'блог'}
1
1
{'центр', 'отзыв', 'крокус', 'закрывать', 'сайт', 'пароль', 'красногорск', 'работа', 'карта', 'россия', 'москва', 'вход', 'входить', 'новость', 'сити', 'поиск', 'регистрация', 'компания', 'вакансия', 'адрес'}
1
{'автор', 'завтра', 'балашов', 'показывать', 'главный', 'день', 'сайт', 'пароль', 'точный', 'прогноз', 'россия', 'входить', 'саратовский', 'новость', 'мир', 'поиск', 'регистрация', 'русский', 'область', 'погода'}
{'растение', 'главный', 'просмотр', 'сайт', 'труд', 'книга', 'борщевик', 'первый', 'помощь', 'вход', 'ожог', 'новость', 'мир', 'поиск', 'страница', 'лечение', 'регистрация', 'форум', 'тема', 'дневник'}
{'зарегистрироваться', 'главный', 'вконтакте', 'сайт', 'пароль', 'питание', 'смородина', 'входить', 'онлайн', 'поиск', 'страница', 'теле

{'солнцезащитный', 'очки', 'выбирать', 'главный', 'сайт', 'линза', 'зрение', 'медицинский', 'пароль', 'оптика', 'контактный', 'входить', 'новость', 'контакт', 'поиск', 'бифокальный', 'регистрация', 'статья', 'форум', 'глаз'}
1
{'главный', 'отзыв', 'инструкция', 'лекция', 'сайт', 'книга', 'пароль', 'ребенок', 'врач', 'входить', 'миопатия', 'поиск', 'страница', 'скачать', 'лечение', 'регистрация', 'дибазол', 'статья', 'форум', 'инфекционный'}
{'образование', 'главный', 'сведение', 'средний', 'бюджетный', 'сайт', 'профессиональный', 'пароль', 'работа', 'государственный', 'организация', 'москва', 'новость', 'колледж', 'поиск', 'страница', 'педагогический', 'учреждение', 'область', 'образовательный'}
{'главный', 'инструкция', 'роутер', 'компьютер', 'вопрос', 'сайт', 'интернет', 'фай', 'сменить', 'пароль', 'поставлять', 'новость', 'телефон', 'узнавать', 'забывать', 'сеть', 'находить', 'программа', 'поменять', 'настройка'}
{'год', 'социальный', 'финансовый', 'главный', 'сайт', 'работа', 'рефе

{'история', 'главный', 'сайт', 'книга', 'пароль', 'работа', 'новость', 'поиск', 'информация', 'искусство', 'переходить', 'страница', 'библиотека', 'греция', 'форум', 'сообщество', 'игра', 'часть', 'древний', 'глава'}
{'осетр', 'язык', 'духовка', 'закрывать', 'здоровье', 'запекать', 'сайт', 'пароль', 'пошаговый', 'кулинарный', 'вход', 'загрузка', 'целиком', 'фото', 'блюдо', 'рыба', 'приготавливать', 'стерлядь', 'регистрация', 'рецепт'}
{'история', 'авторский', 'рассказывать', 'главный', 'гость', 'переменный', 'вопрос', 'сайт', 'пароль', 'работа', 'правовой', 'логин', 'новость', 'поиск', 'страница', 'регистрация', 'форум', 'право', 'друг', 'ответ'}
{'отправлять', 'сохранять', 'закрывать', 'ввод', 'просмотр', 'сайт', 'подождать', 'код', 'симс', 'загрузка', 'полный', 'увеличивать', 'переходить', 'редактирование', 'страница', 'быстрый', 'регистрация', 'предмет', 'игра', 'размер'}
1
1
1
{'спасать', 'главный', 'соль', 'уведомление', 'вопрос', 'гречка', 'исправлять', 'еда', 'кулинарный', 'вход

{'лечить', 'главный', 'день', 'ангина', 'здоровье', 'сайт', 'горло', 'красота', 'ребенок', 'домашний', 'народный', 'условие', 'заболевание', 'контакт', 'болезнь', 'быстро', 'вылечивать', 'лечение', 'средство', 'боль'}
{'год', 'система', 'главный', 'доход', 'экономика', 'книга', 'пароль', 'работа', 'решение', 'вход', 'новость', 'налог', 'поиск', 'налоговый', 'бухгалтерский', 'учет', 'регистрация', 'форум', 'бухгалтерия', 'научный'}
{'обзор', 'главный', 'отзыв', 'инструкция', 'купить', 'техника', 'интернет', 'холодильник', 'пароль', 'стенка', 'входить', 'новость', 'ремонт', 'задний', 'страница', 'бытовой', 'цена', 'форум', 'магазин', 'конкурс'}
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
{'бухгалтер', 'главный', 'закрывать', 'сайт', 'оклад', 'труд', 'пароль', 'работа', 'оплата', 'загрузка', 'новость', 'зарплата', 'страница', 'бухгалтерский', 'учет', 'регистрация', 'форум', 'бухгалтерия', 'компания', 'глава'}
{'номер', 'мтс', 'версия', 'оператор', 'главный', 'клиентамкорпоративный', 'клиентамо', 'тариф

{'магия', 'порча', 'главный', 'заговор', 'сайт', 'книга', 'запись', 'онлайн', 'поиск', 'страница', 'узнавать', 'определять', 'снимать', 'регистрация', 'сервис', 'форум', 'сглаз', 'обсуждение', 'дневник', 'российский'}
{'видео', 'обзор', 'отзыв', 'инструкция', 'язык', 'купить', 'прошивка', 'мобильный', 'планшет', 'объявление', 'пароль', 'ссылка', 'смартфон', 'вход', 'телефон', 'цена', 'регистрация', 'форум', 'прошивать', 'магазин'}
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
{'сервер', 'фотография', 'главный', 'вперед', 'закрывать', 'ввод', 'просмотр', 'карта', 'поздно', 'загрузка', 'запрашивать', 'переходить', 'редактирование', 'загружать', 'скачать', 'попробовать', 'страница', 'предыдущий', 'изменять', 'контент'}
{'закрытие', 'главный', 'ооо', 'адвокат', 'сайт', 'вызываться', 'директор', 'правовой', 'учредительный', 'нужный', 'создание', 'страница', 'открытие', 'услуга', 'учредитель', 'договор', 'регистрация', 'юридический', 'право', 'окно'}
{'главный', 'причина', 'закрывать', 'здоровье', '

In [59]:
scaler = StandardScaler()
scaler.fit(X_test1)
X_test1 = scaler.transform(X_test1)

In [60]:
scaler = StandardScaler()
scaler.fit(X_test4)
X_test4 = scaler.transform(X_test4)

In [61]:
# score на kaggle 0,719
clf = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=5,
                      alpha = 0.01, C=100,batch_size = 100)
clf.fit(X_train3, y_train1)
my_clf_pred = clf.predict(X_test1).reshape(-1,1)
# res1 = (my_clf_pred>0.01*(29))

clf1 = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=6,
                      alpha = 0.01, C=50,batch_size = 100)
clf1.fit(X_train4, y_train1)
my_clf_pred1 = clf1.predict(X_test4).reshape(-1,1)


pred = ((0.54937047*my_clf_pred1+my_clf_pred* 1.05119227)/(0.54937047+1.05119227)).reshape(-1)
print((pred>0.28).sum())
test_data1["target"] = pd.Series(np.array(pred>0.28,dtype = int))
with open("pred_pr_new_28.csv", "w") as f:
    f.write(test_data1.to_csv(columns=("pair_id", "target"), index=False))

/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


5180


In [62]:
y_train1.sum()/y_train1.shape[0]

0.28751069289991443

In [63]:
(pred>0.28).sum()/pred.shape[0]

0.31154146869549526